# Database + GPT

Techniqually its not easy to make GPT fully intellegently utilze the database

In [1]:
from openai import OpenAI
import pymysql
import os

client = OpenAI(api_key=os.getenv("OPENAI_API"))

def get_db_connection():
    return pymysql.connect(
        host='pass-gpt-db.c3oum0k4qlkb.ap-southeast-1.rds.amazonaws.com',
        port=3306,
        user='root',
        password=os.getenv("AWS_IRD_ROOT_KEY"),
        database='passgpt',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )

def clean_sql_query(sql_query):
    # 清理 SQL 语句，移除多余的字符
    sql_query = sql_query.replace("`", "").replace("sql\n", "").replace("\n", " ")
    return ' '.join(sql_query.split())

def sql_query_gpt(user_input):
    connection = get_db_connection()
    
    # 提供数据库结构信息作为系统提示
    database_description = """
    Database 'passgpt' consists of several tables: 'courses', 'files', and 'concepts'.
    - The 'courses' table has columns: course_id (primary key), course_code, course_name, course_description.
    - The 'files' table relates to 'courses' via course_id and includes: file_id (primary key), file_name, title, file_type, file_path, teaching_week, creation_date.
    - The 'concepts' table includes concepts that relate to files and can be hierarchical: concept_id (primary key), parent_id, file_id, concept_name, concept_page, concept_description.
    Relations:
    - 'files' to 'courses' via course_id.
    - 'concepts' to 'files' via file_id and 'concepts' to other 'concepts' via parent_id.
    """
    
    # 初始化系统与用户的对话，包括数据库结构信息
    messages = [
        {"role": "system", "content": database_description},
        {"role": "system", "content": "You are an AI trained to interpret user's input and generate SQL queries based on user's natural language descriptions given the database schema. Please only generate clean SQL queries without explanations."},
        {"role": "user", "content": user_input}
    ]
    
    try:
        # 使用OpenAI的ChatGPT模型处理对话
        chat_completion = client.chat.completions.create(
            model="gpt-4-turbo",  
            messages=messages
        )
        if chat_completion.choices and len(chat_completion.choices) > 0:
            sql_query_cmd = clean_sql_query(chat_completion.choices[0].message.content.strip())
            with connection.cursor() as cursor:
                cursor.execute(sql_query_cmd)
                sql_query_results = cursor.fetchall()
            connection.close()
            return sql_query_cmd, sql_query_results# explanation, sql_query 
        else:
            connection.close()
            return "No response was generated by GPT."
    except Exception as e:
        connection.close()
        return f"Error during the chat completion or SQL execution: {e}"

def course_query_gpt(query, course_info):
    """
    Submits a query along with the formatted course information to GPT for processing,
    using the newer client.chat.completions.create interface.
    """
    messages = [
        {"role": "system", "content": f"{course_info} Your task is to provide a comprehensive response based on the information provided."},
        {"role": "user", "content": query}
    ]
    
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4-turbo",  
            messages=messages,
        )
        if chat_completion.choices and len(chat_completion.choices) > 0:
            last_message = chat_completion.choices[0].message.content.strip()
            return last_message
        else:
            return "No response was returned by GPT."
    except Exception as e:
        return f"Error querying GPT with course info: {e}"
     
# Example use
user_input = "what we have learnt in week 3, any main concepts, and its related sub concepts?"
query_cmd, query_results = sql_query_gpt(user_input)
output = course_query_gpt(user_input, query_results)
print(output)


In week 3, we covered major topics primarily focused on text analysis and dimensionality reduction methods used in data processing and machine learning. Here’s a summary of the main concepts along with their related sub-concepts:

1. **Term Weighting Schemes**
   - **TF-IDF**: Term Frequency-Inverse Document Frequency, a statistical measure used to evaluate the importance of a word to a document in a collection or corpus.
   - **BM25**: A ranking function used by search engines to estimate the relevance of documents to a given search query.

2. **Topic Modeling**
   - **LDA** (Latent Dirichlet Allocation): A generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar.
   - **LSA** (Latent Semantic Analysis): A technique in natural language processing of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms.


# Why don't we integrate them into a single model?
### Advantages:
- Less OPENAI api call cost
- Faster response time
- ...
### Challenges:
- More trial-and-error prompt tuning
- Current OPAI API limit 4096 tokens
- API call can't excute sql query by himself

In [18]:
import os
import pymysql
from openai import OpenAI

class PassGPT:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.database_description = """
        Database 'passgpt' consists of several tables: 'courses', 'files', and 'concepts'.
        - The 'courses' table has columns: course_id (primary key), course_code, course_name, course_description.
        - The 'files' table relates to 'courses' via course_id and includes: file_id (primary key), file_name, title, file_type, file_path, teaching_week, creation_date.
        - The 'concepts' table includes concepts that relate to files and can be hierarchical: concept_id (primary key), parent_id(NULL for top concepts), file_id, concept_name, concept_page, concept_description.
        Relations:
        - 'files' to 'courses' via course_id.
        - 'concepts' to 'files' via file_id and 'concepts' to other 'concepts' via parent_id(hierarchical).
        """
    
    def get_db_connection(self):
        return pymysql.connect(
            host='pass-gpt-db.c3oum0k4qlkb.ap-southeast-1.rds.amazonaws.com',
            port=3306,
            user='root',
            password=os.getenv("AWS_IRD_ROOT_KEY"), # Replace with your own AWS RDS root password
            database='passgpt',
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
    
    def clean_sql_query(self, sql_query):
        sql_query = sql_query.replace("`", "").replace("sql\n", "").replace("\n", " ")
        return ' '.join(sql_query.split())

    def execute_sql_query(self, sql_query):
        connection = self.get_db_connection()
        try:
            with connection.cursor() as cursor:
                cursor.execute(sql_query)
                return cursor.fetchall()
        finally:
            connection.close()

    def generate_and_process_query(self, user_input):
        prompt = f"{self.database_description} Based on the database schema, generate SQL queris to retrieve all relevant data with detailed information, based on the user input: '{user_input}'. 
            Then, your need to provide a comprehensive response based on the information provided, describe the results in a detailed, human-readable form for education need.
            You may need to provide a detailed explanation of the results, including the main concepts and related sub-concepts.
            If you are asked to provide quiz/question/test, for code questions, the questions should be single choice questions testing understanding of the code representing related concepts, you should also provide the code segment;for not specified code questions, the questions should have multiple choice questions testing understanding of the terms. Finally, you should give the answer"

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ]
        try:
            chat_completion = self.client.chat.completions.create(
                model="gpt-4-turbo",  
                messages=messages
            )
            if chat_completion.choices and len(chat_completion.choices) > 0:
                #TODO: this is where we can make it more integrate into GPT
                sql_query = self.clean_sql_query(chat_completion.choices[0].message.content.strip())
                results = self.execute_sql_query(sql_query)
                return results
            else:
                return "No response was generated by GPT."
        except Exception as e:
            return f"Error during the chat completion or SQL execution: {e}"

# Example usage:
api_key = os.getenv("OPENAI_API")
db_query_processor = PassGPT(api_key=api_key)
user_input = "what we have learnt in week 3, any main concepts, in which page?"
response = db_query_processor.generate_and_process_query(user_input)
print(response)
